In [2]:
# make_dataset_gnn_ps.py
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import os
from pipeline.modules import CambModule, ClassModule, EmceeModule
from pipeline.functions import log_posterior

In [3]:
pk_read = pd.read_csv('../Data/std-camb-data.csv')
pk_std = pk_read[['pk']]

cosmo = [67.4, 0.02237, 0.12, 0.06, 2.10e-9, 0.965]

camb_mod = CambModule(fixed_params={"cosmo_params": cosmo},
                      var_params={"AccuracyBoost": list(np.linspace(1.0, 3.0, 3)),
                                  "lAccuracyBoost": list(np.linspace(1.0, 3.0, 3)),
                                  "lSampleBoost": list(np.linspace(1.0, 3.0, 3))}
                     )

class_mod = ClassModule(fixed_params={"cosmo_params": cosmo},
                        var_params={"tol_background": list(np.linspace(1e-6, 1e-10, 3)),
                                    "tol_thermo": list(np.linspace(1e-4, 1e-6, 3))}
                       )

mean = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

# 6x6 covariance matrix (symmetric, positive-definite)
cov = np.array([
    [1.0, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.2, 1.5, 0.2, 0.1, 0.0, 0.0],
    [0.1, 0.2, 1.2, 0.1, 0.0, 0.0],
    [0.0, 0.1, 0.1, 1.0, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.1, 0.8, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.9]
])
initial_guess = np.zeros(6)

Input = {"mean": mean,"cov": cov}

# std_df = pd.DataFrame({"mu" : mean, "sigma" : np.sqrt(np.diag(cov))})
std_df = {"mean": mean}

emcee_mod = EmceeModule(fixed_params={"initial_guess": initial_guess,
                                      "loglike_func": log_posterior,
                                      "like_args": Input},
                        var_params={"n_walkers": list(np.linspace(50, 500, 5).astype(int)),
                                    "n_steps": list(np.linspace(200, 5000, 5).astype(int))})



In [4]:
# 27 min 35 sec for 10201 samples
class_dataset = class_mod.run_var_params(std_output=pk_std, mpi=8)
class_dataset.to_csv('gnn_class_dataset.csv', index=None)

No. of workers used = 8


CLASS MPI: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  4.13it/s]


In [5]:
camb_dataset = camb_mod.run_var_params(std_output=pk_std, mpi=8)
camb_dataset.to_csv('gnn_camb_dataset.csv', index=None)

No. of workers used = 8


CAMB MPI: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:54<00:00,  2.03s/it]


In [ ]:
dataset = emcee_mod.run_var_params(std_output=std_df, mpi=8)
dataset.to_csv('gnn_emcee_dataset.csv', index=None)

In [6]:
df_run = camb_mod.run()


In [ ]:
import itertools
keys = list(camb_mod.var_params.keys())
values = [camb_mod.var_params[k] for k in keys]
combos = list(itertools.product(*values))

param_set = dict(zip(keys, combos[0]))
camb_mod.run(**param_set)

In [ ]:
type(pk_std)

In [7]:
camb_mod.compare_func(df_run['result'], pk_std)

(-1.6324786932937794e-05, 0.00023667073219173874)

In [ ]:
camb_mod.default_compare(df_run['result'], pk_std)

In [ ]:
rel = (df_run['result'].iloc[:,0] - pk_read[['pk']].iloc[:,0])/(np.abs(pk_read[['pk']].iloc[:,0]) + 1.0e-16)
rel.std()

In [ ]:
rel(